# Step 1: annotazione

Frame OLIVERio:
[x] Concessive
[x] History
[x] Change_resistance
[x] Emptying
[x] Performers_and_roles

Frame TOMATIS
[x] Deciding
[x] Intentionally_act
[x] Competition
[x] Fairness_evaluation
[x] Process_continue

In [1]:
from nltk.corpus import framenet as fn
from nltk.corpus import wordnet as wn

frame = fn.frame_by_name('Performers_and_roles')

print('definizione frame:', frame.definition)
print('\n')

for fe in frame.FE:
    print(fe, ': ', frame.FE[fe].definition)

print('\n')
for lu in frame.lexUnit:
    print(lu, ': ', frame.lexUnit[lu].definition)

definizione frame: A Performer has a certain Role in a Performance and plays their part by following the directions in the Script for that particular Role.   'Margaret Thatcher played Juliet in a school play.' '"Pulp Fiction" starred John Travolta and a lot of other big names.' 'Mel Gibson starred in "Braveheart" as the leader of a Scottish rebellion.' 'I play the character of Yusuf in this film.'




star.v :  COD: have (someone) as a principal performer. 
act.v :  COD: perform a fictional role in a play or film. 
play.v :  COD: . represent (a character) in a play or film
be.v :  FN: perform in the role of
star.n :  FN: an important performer
co-star.n :  COD: a performer appearing with another or others of equal importance. 
co-star (in).v :  COD:  appear in a production as a co-star.
lead.n :  COD: the chief part in a play or film. 
star (in).v :  COD: have a principal role in a film, play, etc. 
as.prep :  FN: playing in the role of
co-star.v :  COD: include as a co-star. 
feature.

In [7]:
print('\n')
for syn in wn.synsets('lead'):
    print(syn)
    print('definition:' , syn.definition())
    #print example
    for example in syn.examples():
        print(example)

    print('\n')



Synset('lead.n.01')
definition: an advantage held by a competitor in a race
he took the lead at the last turn


Synset('lead.n.02')
definition: a soft heavy toxic malleable metallic element; bluish white when freshly cut but tarnishes readily to dull grey
the children were playing with lead soldiers


Synset('lead.n.03')
definition: evidence pointing to a possible solution
the police are following a promising lead
the trail led straight to the perpetrator


Synset('lead.n.04')
definition: a position of leadership (especially in the phrase `take the lead')
he takes the lead in any group
we were just waiting for someone to take the lead
they didn't follow our lead


Synset('lead.n.05')
definition: the angle between the direction a gun is aimed and the position of a moving target (correcting for the flight time of the missile)


Synset('lead.n.06')
definition: the introductory section of a story
it was an amusing lead-in to a very serious matter


Synset('lead.n.07')
definition: (sports

# Step 2: mapping automatico

## Approccio bag of words

In [45]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer #tiene conto delle multiword expressions
from nltk.corpus import wordnet as wn
from nltk.corpus import framenet as fn
import re


In [46]:
deciding_frame = fn.frame_by_name("Deciding")

In [47]:
#creazione tokenizzatore per le multiword expressions
mwes = [x for x in wn.all_lemma_names() if '_' in x]
mwes = [tuple(x.split('_')) for x in mwes]
tokenizer = MWETokenizer(mwes, separator=' ')

def make_set(sentence):
    sentence = sentence.lower() #lowercase
    sentence = re.sub(r'[^\w\s]','',sentence) #remove punctuation
    sentence = tokenizer.tokenize(sentence.split()) #tokenize
    sentence = [w for w in sentence if not w.isdigit()] #remove numbers
    stop_words = set(stopwords.words('english')) #remove stop words
    sentence = [w for w in sentence if not w in stop_words]
    lemmatizer = WordNetLemmatizer() #lemmatization of definition
    sentence = [lemmatizer.lemmatize(w) for w in sentence]

    res = []
    for w in sentence:
        res.append(w.replace(' ', '_'))

    return set(res)

make_set("I'm going to the park of New York.")

{'going', 'im', 'new_york', 'park'}

In [48]:
def ctx_frame_FN(frame):
    name = frame.name
    definition = frame.definition
    FEs = frame.FE
    LUs = frame.lexUnit

    sentence = name + ' ' + definition
    for fe in FEs:
        sentence += ' ' + FEs[fe].definition
    for lu in LUs:
        sentence += ' ' + LUs[lu].definition

    return make_set(sentence)

def ctx_frame_element_FN(frame_element):
    name = frame_element.name
    definition = frame_element.definition
    #semtype = frame_element.semType.name

    sentence = name + ' ' + definition # + ' ' + semtype
   
    return make_set(sentence)

def ctx_lexical_unit_FN(lexical_unit):
    name = lexical_unit.name
    definition = lexical_unit.definition.split(':')[1]
    exemplars = lexical_unit.exemplars
    
    sentence = name + ' ' + definition
    for ex in exemplars:
        sentence += ' ' + ex.annotationSet[0].text

    return make_set(sentence)


### Creazione del contesto (FrameNet)

In [49]:
ctx_FN = {}

#frame
ctx_FN[deciding_frame.name.replace(' ', '_')] = ctx_frame_FN(deciding_frame)

#FEs
for fe in deciding_frame.FE:
    ctx_FN[fe.replace(' ', '_')] = ctx_frame_element_FN(deciding_frame.FE[fe])
    break

#LUs
for lu in deciding_frame.lexUnit:
    ctx_FN[lu.replace(' ', '_')] = ctx_lexical_unit_FN(deciding_frame.lexUnit[lu])


### Crezione del contesto (WordNet)

In [50]:
def get_hyponyms(synset):
    hyponyms = set()
    for hyponym in synset.hyponyms():
        hyponyms |= set(get_hyponyms(hyponym))
    return hyponyms | set(synset.hyponyms())

In [51]:
def create_sentences(synset, depth=1):
    sentence = synset.definition()
    for example in synset.examples():
        sentence += ' ' + example
    for lemma in synset.lemmas():
        sentence += ' ' + lemma.name()

    if (depth >= 0):
        for hypernym in synset.hypernyms():
            sentence += ' ' + create_sentences(hypernym, depth-1)
        for hyponym in get_hyponyms(synset):
            sentence += ' ' + create_sentences(hyponym, depth-1)  

    return sentence


def ctx_synset_WN(synset):
    sentence = create_sentences(synset)
    return make_set(sentence)

In [52]:
mappings = {}
for key in ctx_FN:
    token = key.split('.')[0]
    syns = wn.synsets(token)
    max_overlap = 0
    for syn in syns:
        overlap = len(ctx_synset_WN(syn).intersection(ctx_FN[key])) + 1
        if overlap > max_overlap:
            max_overlap = overlap
            mappings[key] = syn

mappings

{'Deciding': Synset('decision_making.n.01'),
 'decide.v': Synset('decide.v.02'),
 'decision.n': Synset('decision.n.01'),
 'rule_out.v': Synset('rule_out.v.02'),
 'determine.v': Synset('determine.v.02')}

## Approccio grafico

In [53]:
deciding_frame = fn.frame_by_name("Deciding")

In [54]:
#FEs
for fe in deciding_frame.FE: 
    ctx_FN[fe.replace(' ', '_')] = ctx_frame_element_FN(deciding_frame.FE[fe])
#LUs
for lu in deciding_frame.lexUnit:
    ctx_FN[lu.replace(' ', '_')] = ctx_lexical_unit_FN(deciding_frame.lexUnit[lu])

Cognizer
Decision
Possibilities
Time
Place
Manner
Inherent_purpose
Circumstance
Explanation


In [55]:
def get_paths_between_synsets(synset1, synset2, L=3):
    paths = []
    visited = set()

    def dfs(synset, path):
        if synset in visited or len(path) > L:
            return
        if synset == synset2:
            paths.append(path + [synset])
            return
        visited.add(synset)
        for hypernym in synset.hypernyms():
            dfs(hypernym, path + [synset])
        for hyponym in synset.hyponyms():
            dfs(hyponym, path + [synset])

    dfs(synset1, [])
    return [path for path in paths if len(path) <= L]

In [56]:
import numpy as np

def score(syn_FN, word_FN, ctx_FN):
    ctx_word = ctx_FN[word_FN]

    res = 0
    for word in ctx_word:
        for syn in wn.synsets(word):
            #get all connection path between syn and syn_FN
            paths = list(get_paths_between_synsets(syn, syn_FN, 3))

            for path in paths:
                res += np.exp(-len(path)-1)

    return res

def prob(syn_FN, word_FN, ctx_FN):
    sum = 0
    for key in ctx_FN:
        for syn in wn.synsets(key):
            sum += score(syn, key, ctx_FN)

    return score(syn_FN, word_FN, ctx_FN) / sum

def argmax_prob(word_FN, ctx_FN):
    max_prob = 0
    max_syn = None
    for syn in wn.synsets(word_FN):
        prob_syn = prob(syn, word_FN, ctx_FN)
        if prob_syn > max_prob:
            max_prob = prob_syn
            max_syn = syn
            
    return max_syn

print(wn.synsets('deciding'))

argmax_prob('Deciding', ctx_FN)

[Synset('decision_making.n.01'), Synset('decide.v.01'), Synset('decide.v.02'), Synset('decide.v.03'), Synset('decide.v.04'), Synset('deciding.s.01')]


Synset('decide.v.01')

# Step 3: valutazione

In [ ]:
# to do...